In [ ]:
!pip install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 59.2 MB/s eta 0:00:00


In [ ]:
import pulp as plp
import numpy as np

In [ ]:
prob = plp.LpProblem("Sudoku_Solver")

# Set the objective function
# Sudoku works only on the constraints
# There is no objective function that we are trying maximize or minimize.
# Set a dummy objective

objective = plp.lpSum(0)
prob.setObjective(objective)

rows = range(0,9)
cols = range(0,9)
grids = range(0,9)
values = range(1,10)

# Decision Variable/Target variable
grid_vars = plp.LpVariable.dicts("grid_value", (rows,cols,values), cat='Binary')


In [ ]:
# Regras
# CONSTRAINT 1: Constraint to ensure only one value is filled for a cell
for row in rows:
    for col in cols:
            # print([grid_vars[row][col][value] for value in values])
            prob.addConstraint(plp.LpConstraint(e=plp.lpSum([grid_vars[row][col][value] for value in values]),
                                    sense=plp.LpConstraintEQ, rhs=1, name=f"constraint_sum_{row}_{col}"))

# CONSTRAINT 2: Constraint to ensure that values from 1 to 9 is filled only once in a row
for row in rows:
    for value in values:
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([grid_vars[row][col][value]*value  for col in cols]),
                                    sense=plp.LpConstraintEQ, rhs=value, name=f"constraint_uniq_row_{row}_{value}"))

# CONSTRAINT 3: Constraint to ensure that values from 1 to 9 is filled only once in a column
for col in cols:
    for value in values:
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([grid_vars[row][col][value]*value  for row in rows]),
                                    sense=plp.LpConstraintEQ, rhs=value, name=f"constraint_uniq_col_{col}_{value}"))

# CONSTRAINT 4: Constraint to ensure that values from 1 to 9 is filled only once in the 3x3 grid
for grid in grids:
    grid_row  = int(grid/3)
    grid_col  = int(grid%3)

    for value in values:
        prob.addConstraint(plp.LpConstraint(e=plp.lpSum([grid_vars[grid_row*3+row][grid_col*3+col][value]*value for col in range(0,3) for row in range(0,3)]),
                                    sense=plp.LpConstraintEQ, rhs=value, name=f"constraint_uniq_grid_{grid}_{value}"))



In [ ]:
#Setup Incial
input_sudoku = [
                    [0,0,0,0,0,0,0,0,9],
                    [0,4,0,8,3,0,0,0,0],
                    [8,2,0,0,4,0,0,1,0],
                    [0,0,6,0,0,7,0,0,0],
                    [0,0,8,0,0,0,5,0,7],

                    [0,5,3,2,0,0,0,0,0],
                    [0,7,0,9,2,0,0,0,5],
                    [0,0,0,0,0,0,0,0,0],
                    [1,0,5,0,6,0,0,0,0]
                ]

# Fill the prefilled values from input sudoku as constraints
for row in rows:
    for col in cols:
        if(input_sudoku[row][col] != 0):
            prob.addConstraint(plp.LpConstraint(e=plp.lpSum([grid_vars[row][col][value]*value  for value in values]),
                                    sense=plp.LpConstraintEQ,
                                    rhs=input_sudoku[row][col],
                                    name=f"constraint_prefilled_{row}_{col}"))

In [ ]:
prob.solve()

1

In [ ]:
print(f'Solution Status = {plp.LpStatus[prob.status]}')

# Code to extract the final solution grid
solution = [[0 for col in cols] for row in rows]
grid_list = []
for row in rows:
    for col in cols:
        for value in values:
            if plp.value(grid_vars[row][col][value]):
                solution[row][col] = value

# Print the final solution as a grid
print(f"\nFinal result:")

print("\n\n+ ----------- + ----------- + ----------- +",end="")
for row in rows:
    print("\n",end="\n|  ")
    for col in cols:
        num_end = "  |  " if ((col+1)%3 == 0) else "   "
        print(solution[row][col],end=num_end)

    if ((row+1)%3 == 0):
        print("\n\n+ ----------- + ----------- + ----------- +",end="")

Solution Status = Optimal

Final result:


+ ----------- + ----------- + ----------- +

|  5   3   1  |  6   7   2  |  4   8   9  |  

|  6   4   9  |  8   3   1  |  7   5   2  |  

|  8   2   7  |  5   4   9  |  3   1   6  |  

+ ----------- + ----------- + ----------- +

|  4   9   6  |  1   5   7  |  2   3   8  |  

|  2   1   8  |  3   9   6  |  5   4   7  |  

|  7   5   3  |  2   8   4  |  6   9   1  |  

+ ----------- + ----------- + ----------- +

|  3   7   4  |  9   2   8  |  1   6   5  |  

|  9   6   2  |  4   1   5  |  8   7   3  |  

|  1   8   5  |  7   6   3  |  9   2   4  |  

+ ----------- + ----------- + ----------- +

Solution Status = Optimal

Final result:


+ ----------- + ----------- + ----------- +

|  3   6   7  |  8   9   4  |  2   5   1  |  

|  5   9   8  |  3   1   2  |  6   7   4  |  

|  2   4   1  |  5   7   6  |  8   3   9  |  

+ ----------- + ----------- + ----------- +

|  7   2   3  |  9   8   1  |  4   6   5  |  

|  8   5   6  |  4   2   3  |  9   1   7  |  

|  4   1   9  |  7   6   5  |  3   2   8  |  

+ ----------- + ----------- + ----------- +

|  1   8   5  |  6   3   9  |  7   4   2  |  

|  6   7   2  |  1   4   8  |  5   9   3  |  

|  9   3   4  |  2   5   7  |  1   8   6  |  

+ ----------- + ----------- + ----------- +
